In [69]:
import mysql.connector
import os
import pandas as pd

def connect_to_database(host='127.0.0.1', port=3306, user='root', password='Xhypp*426118', database='proj'):
    """
        Connect to a MySQL database and return the connection and cursor.
        :param host: The host IP address or domain name.
        :param port: The port number.
        :param user: The username.
        :param password: The password.
        :param database: The database name.
        :return: The connection and cursor.
    """
    conn = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    return conn, cursor

def create_table(cursor, sql):
    """
        Create a table.
        :param cursor: The cursor object.
        :param sql: The SQL statement.
    """
    cursor.execute(sql)

def insert_data(cursor, sql, data):
    """
        Insert data into a table.
        :param cursor: The cursor object.
        :param sql: The SQL statement.
        :param data: The data to be inserted.
    """
    cursor.execute(sql, data)

def query_data(cursor, sql):
    """
        Query data from a table.
        :param cursor: The cursor object.
        :param sql: The SQL statement.
        :return: The query results.
    """
    cursor.execute(sql)
    return cursor.fetchall()

def customer_order(cursor, id):
    op = f"""WITH tmp AS (SELECT * FROM `order` WHERE customer_id = {id})
    SELECT * FROM tmp, customer
    WHERE tmp.customer_id = customer.customer_id"""
    cursor.execute(op)
    return cursor.fetchall()

def execute_query_from_file(cursor, file_path, params=None):
    '''
        Execute SQL commands from a file
        :param cursor: The cursor object.
        :param file_path: The path of the SQL file.
        :param params: The parameters to be passed to the SQL file.
    '''

    with open(file_path, 'r') as file:
        sql_script = file.read()
    
    if params:
        sql_script = sql_script % params
    sql_commands = sql_script.split(';')
    for command in sql_commands:
        if command.strip():
            try:
                cursor.execute(command)
            except mysql.connector.Error as e:
                print(f"An error occurred: {e}")

    
    results = cursor.fetchall()  
    return results


def close_connection(conn):
    """
        Close the connection.
        :param conn: The connection object.
    """
    conn.commit()
    conn.close()

def run_sql_file(filename, conn):
    """
        Run an SQL file.
        :param filename: The filename of the SQL file.
        :param conn: The connection object.
    """
    with open(filename, 'r', encoding='utf8') as file:
        sql_script = file.read()
    
    sql_commands = sql_script.split(';')

    for command in sql_commands:
        if command.strip():
            cursor = conn.cursor()  
            try:
                cursor.execute(command)
                conn.commit()  
            except mysql.connector.Error as e:
                print(f"An error occurred: {e}")
            finally:
                cursor.close() 

host = '127.0.0.1'
port = 3306
user = 'youmeng'
password = '123456'   
database = 'proj'
buffered = True

# Connect to the database
conn, cursor = connect_to_database(host, port, user, password, database)
current_directory = os.getcwd() 

In [67]:
def excel_to_sql_query(filename, table_name):
    # 读取CSV文件
    data = pd.read_excel(filename)
    columns = ', '.join(data.columns)
    sql_query = f"INSERT INTO {table_name} ({columns}) VALUES "
    for ite, row in data.iterrows():
        row = tuple(row)
        sql_query += str(row) + ','
    sql_query = sql_query[:-1] + ';'
    return sql_query


In [68]:
current_directory = os.getcwd()
table_name = 'customer'
filename = os.path.join(current_directory, '', f'{table_name}.xlsx')
excel_to_sql_query(filename, table_name)

"INSERT INTO customer (customer_id, record_id, customer_name, customer_gender, customer_birth, customer_phone, customer_password) VALUES (1, 1, 'John Doe', 'Male', Timestamp('1990-05-15 00:00:00'), '123-456-7890', 'password123'),(2, 2, 'Jane Smith', 'Female', Timestamp('1988-09-20 00:00:00'), '987-654-3210', 'abc123'),(3, 3, 'David Lee', 'Male', Timestamp('1995-12-10 00:00:00'), '555-555-5555', 'qwerty');"

In [48]:
import pandas as pd

for table_name in ['customer', 'order', 'item', 'sign_up_record', 'store', 'order_item']:
    filename = os.path.join(current_directory, '', f'{table_name}.xlsx')
    data = pd.read_excel(filename)
    data.columns = [col.lower() for col in data.columns]
    data.to_excel(filename, index=False)
    print(data)

   customer_id  record_id customer_name customer_gender customer_birth  \
0            1          1      John Doe            Male     1990-05-15   
1            2          2    Jane Smith          Female     1988-09-20   
2            3          3     David Lee            Male     1995-12-10   

  customer_phone customer_password  
0   123-456-7890       password123  
1   987-654-3210            abc123  
2   555-555-5555            qwerty  
   order_id  store_id  customer_id          order_time order_status
0         1         1            1 2024-04-20 10:00:00    Delivered
1         2         2            2 2024-04-19 15:30:00      Pending
2         3         3            3 2024-04-18 09:45:00    Completed
    item_id  store_id        item_name  item_price (rmb)    item_type  \
0         1         1        Coca-Cola               3.0     Beverage   
1         2         1    Lay's Classic               5.5        Snack   
2         3         1  Instant Noodles               6.8        

## item_detail

In [75]:
filename = os.path.join(current_directory.replace('data','query'), '', 'item_detail.sql')
res = execute_query_from_file(cursor, filename,('Chocolate',))
df = pd.DataFrame(res)
df.columns = ['item_name', 'store_name', 'store_id', 'store_address', 'item_id', 'item_price', 'item_type', 'item_inventory']
df

,item_name,store_name,store_id,store_address,item_id,item_price,item_type,item_inventory
0,KitKat,711,1,Upper Campus,4,4.20,Chocolate,35
1,Mars Bar,711,1,Upper Campus,10,3.80,Chocolate,40
2,Snickers,711,1,Upper Campus,12,4.50,Chocolate,30
3,Twix,711,1,Upper Campus,16,4.20,Chocolate,35
4,Reese's Peanut Butter Cups,711,1,Upper Campus,19,6.80,Chocolate,25
5,Hershey's,711,1,Upper Campus,21,4.50,Chocolate,35
6,M&M's,711,1,Upper Campus,25,5.50,Chocolate,30
7,Hershey's Kisses,711,1,Upper Campus,28,6.80,Chocolate,25


## order_detail

In [76]:
filename = os.path.join(current_directory.replace('data','query'), '', 'order_detail.sql')
res = execute_query_from_file(cursor, filename,(1,))
df = pd.DataFrame(res)
df.columns = ["order_id", "item_name", "item_id", "store_id", "quantity", "customer_id", "order_time", "order_status", "item_price", "item_type", "item_inventory"]
df

,order_id,item_name,item_id,store_id,quantity,customer_id,order_time,order_status,item_price,item_type,item_inventory
0,1,Mars Bar,10,1,2,1,2024-04-20 10:00:00,Delivered,3.80,Chocolate,40
1,1,Pocky,13,1,3,1,2024-04-20 10:00:00,Delivered,5.80,Snack,50
2,1,M&M's,25,1,1,1,2024-04-20 10:00:00,Delivered,5.50,Chocolate,30


## customer_order

In [77]:
filename = os.path.join(current_directory.replace('data','query'), '', 'customer_order.sql')
res = execute_query_from_file(cursor, filename,(1,))
df = pd.DataFrame(res)
df.columns = ['customer_id', 'order_id', 'store_name', 'order_time', 'order_status']
df

,customer_id,order_id,store_name,order_time,order_status
0,1,1,711,2024-04-20 10:00:00,Delivered


## store_detail

In [86]:
filename = os.path.join(current_directory.replace('data','query'), '', 'store_detail.sql')
res = execute_query_from_file(cursor, filename,(1,))
df = pd.DataFrame(res)
df.columns = ['item_id', 'item_name','item-price','item_type','item_inventory']
df

,item_id,item_name,item-price,item_type,item_inventory
0,1,Coca-Cola,3.00,Beverage,50
1,2,Lay's Classic,5.50,Snack,40
2,3,Instant Noodles,6.80,Food,60
3,4,KitKat,4.20,Chocolate,35
4,5,Pringles,7.00,Snack,25
5,6,Red Bull,8.50,Energy Drink,30
6,7,Bottled Water,2.50,Beverage,70
7,8,Oreo,4.80,Biscuit,45
8,9,Doritos,6.00,Snack,20
9,10,Mars Bar,3.80,Chocolate,40
